#### **What are Embeddings?**
Embeddings are high-dimensional vector representations of text. They allow you to encode the meaning of text in a way that a machine can compare and retrieve semantically similar content.

In a RAG pipeline, embeddings help match user queries with the most relevant document chunks based on semantic similarity, not just keyword matching.

#### **Embedding Step in LangChain (Deep Dive)**
In LangChain, the embedding step follows text loading and text splitting, and comes before storing or retrieving from a vector database.

#### **Key Concepts**
1. Text Input: Usually a list of strings (chunks of documents).
2. Embedding Model: Transforms text into vectors.
3. Output: A list of vectors (float arrays).



**Embedding Models in LangChain**: LangChain abstracts embedding models behind a standard interface using Embeddings class:

| Provider                               | LangChain Class         | Notes                              |
| -------------------------------------- | ----------------------- | ---------------------------------- |
| **OpenAI**                             | `OpenAIEmbeddings`      | Uses `text-embedding-ada-002`      |
| **Hugging Face**                       | `HuggingFaceEmbeddings` | Local or hosted transformers       |
| **Cohere**                             | `CohereEmbeddings`      | Fast and scalable                  |
| **Google Vertex AI**                   | `VertexAIEmbeddings`    | Google’s models                    |
| **Azure OpenAI**                       | `AzureOpenAIEmbeddings` | Same as OpenAI but hosted on Azure |
| **Local** (e.g. Sentence Transformers) | `HuggingFaceEmbeddings` | No API call, great for privacy     |


**Choosing Right Embedding Model**:
| Use Case                  | Suggested Embedding Model     |
| ------------------------- | ----------------------------- |
| General-purpose (cloud)   | `OpenAIEmbeddings`            |
| Open-source/local         | `HuggingFaceEmbeddings`       |
| Privacy-sensitive         | Sentence Transformers (local) |
| High-scale commercial use | `CohereEmbeddings`            |


**Tips for Better Embedding Usage**:
1. Clean and chunk text well before embedding.
2. Avoid embedding too-long texts; keep chunks ≤ 1000 tokens.
3. Ensure the same embedding model is used for both indexing and querying.
4. Consider using embedding caching for speed.


In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model= 'text-embedding-3-large')

embeddings

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x000001ED502C1C60>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x000001ED5075CD60>, model='text-embedding-3-large', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [3]:
from langchain.embeddings import HuggingFaceEmbeddings

model_name = 'sentence-transformers/all-MiniLM-L6-v2'  # Free and light
embeddings = HuggingFaceEmbeddings(model_name=model_name)

embeddingstext = 'i am physics undergraduate student'
query_result = embeddings.embed_query(embeddingstext)

print(query_result[:5])  # Print first 5 dimensions


C:\Users\disha\AppData\Local\Temp\ipykernel_10856\4128701652.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=model_name)
c:\Users\disha\miniconda3\envs\rag-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[-0.043438367545604706, -0.02323009818792343, -0.019097167998552322, 0.02411555126309395, -0.0035267199855297804]


In [4]:
len(query_result)

384

In [5]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader('quantum.txt')
docs = loader.load()
docs

[Document(metadata={'source': 'quantum.txt'}, page_content='Quantum Mechanics: An Overview\nQuantum mechanics is a fundamental branch of physics that describes nature at the smallest scales of energy levels of atoms and subatomic particles. Unlike classical mechanics, where objects have definite positions and velocities, quantum mechanics introduces a probabilistic framework. Particles are described by wave functions, and these wave functions provide the probabilities of finding a particle in a particular location or state.\n\nWave-Particle Duality\nOne of the most profound revelations of quantum mechanics is that particles, such as electrons and photons, exhibit both wave-like and particle-like properties. This dual nature was highlighted in the famous double-slit experiment, where particles produce an interference pattern typical of waves when not observed, but behave like discrete particles when measured.\n\nThe Uncertainty Principle\nProposed by Werner Heisenberg, the uncertainty p

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50)
final_documents=text_splitter.split_documents(docs)
final_documents

[Document(metadata={'source': 'quantum.txt'}, page_content='Quantum Mechanics: An Overview\nQuantum mechanics is a fundamental branch of physics that describes nature at the smallest scales of energy levels of atoms and subatomic particles. Unlike classical mechanics, where objects have definite positions and velocities, quantum mechanics introduces a probabilistic framework. Particles are described by wave functions, and these wave functions provide the probabilities of finding a particle in a particular location or state.'),
 Document(metadata={'source': 'quantum.txt'}, page_content='Wave-Particle Duality\nOne of the most profound revelations of quantum mechanics is that particles, such as electrons and photons, exhibit both wave-like and particle-like properties. This dual nature was highlighted in the famous double-slit experiment, where particles produce an interference pattern typical of waves when not observed, but behave like discrete particles when measured.'),
 Document(metad

In [7]:
## Vector Embedding And Vector StoreDB
from langchain_community.vectorstores import Chroma

db=Chroma.from_documents(final_documents,embeddings)
db



In [8]:
### Retrieve the results from query vectorstore db
query="It will be all the easier for us to conduct ourselves as belligerents"
retrieved_results=db.similarity_search(query)
print(retrieved_results)

[Document(metadata={'source': 'quantum.txt'}, page_content="Quantum Computing\nQuantum computing harnesses the power of qubits, which can exist in superpositions of states. Quantum algorithms such as Shor's (for factoring large numbers) and Grover's (for search problems) demonstrate potential exponential speedups over classical counterparts. While still in its early stages, quantum computing could revolutionize fields from cryptography to drug discovery."), Document(metadata={'source': 'quantum.txt'}, page_content='Applications of Quantum Mechanics\nQuantum mechanics has led to revolutionary technologies, including semiconductors, lasers, magnetic resonance imaging (MRI), and quantum computers. Quantum cryptography, using principles like entanglement and superposition, offers the promise of unbreakable encryption.'), Document(metadata={'source': 'quantum.txt'}, page_content='Measurement Problem and Interpretations\nThe measurement problem arises from the transition of a quantum system 

### **OLAMA**
Ollama supports embedding models, making it possible to build retrieval augmented generation (RAG) applications that combine text prompts with existing documents or other data.

In [12]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings =(
    OllamaEmbeddings(model= 'nomic-embed-text')
)

In [13]:
embeddings

OllamaEmbeddings(base_url='http://localhost:11434', model='nomic-embed-text', embed_instruction='passage: ', query_instruction='query: ', mirostat=None, mirostat_eta=None, mirostat_tau=None, num_ctx=None, num_gpu=None, num_thread=None, repeat_last_n=None, repeat_penalty=None, temperature=None, stop=None, tfs_z=None, top_k=None, top_p=None, show_progress=False, headers=None, model_kwargs=None)

In [14]:
r1=embeddings.embed_documents(
    [
       "Alpha is the first letter of Greek alphabet",
       "Beta is the second letter of Greek alphabet", 
    ]
)

### Hugging face embeddings